# re

re - это модуль Python для работы с PCRE-совместимым регулярными выражениями.

> Регуля́рные выраже́ния (англ. regular expressions) — формальный язык поиска и осуществления манипуляций с подстроками в тексте, основанный на использовании метасимволов (символов-джокеров, англ. wildcard characters). Для поиска используется строка-образец (англ. pattern, по-русски её часто называют «шаблоном», «маской»), состоящая из символов и метасимволов и задающая правило поиска. Для манипуляций с текстом дополнительно задаётся строка замены, которая также может содержать в себе специальные символы.

In [79]:
import re  # импортируем модуль

### Разберем конкретные примеры

#### 1. Проверить является ли строка email адресом: `.match()`

In [80]:
emails = [
  "foobar.com",  # not ok
  "foo@bar.com",  # ok
  "foo@barcom",  # not ok
  "foo@bar.com.ru.net",  # ok
]

In [81]:
regex = r"^.*?@.*?\..*?$"  # неидеальное выражение

# выражение получше, но тоже несовершенное
# r"(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"

Строки с регулярными выражениями маркиурются с помощью `r`.

Это позволяет удобно задавать регулярные выражения, избегая проблем с интерполяцией.

In [82]:
for email in emails:
  matches = re.match(regex, email)  # None если нет совпадений, иначе объект Match
  if matches is None:
    print(f"Строка \"{email}\" - не похожа на email.")
  else:
    print(f"А вот это уже email: \"{email}\".")


Строка "foobar.com" - не похожа на email.
А вот это уже email: "foo@bar.com".
Строка "foo@barcom" - не похожа на email.
А вот это уже email: "foo@bar.com.ru.net".


#### 2. Найти время в строке. Например: "Поужинать в `09:00` и встретится с Бобом в `13:00`": `.findall()`

In [83]:
string = "Поужинать в 09:00 и встретится с Бобом в 13:00"

In [84]:
regex = r"(?:2[0-3]|[0-1]\d):[0-5]\d"  # регулярное выражение

times_pattern = re.compile(regex)  # компилируем регулярное выражение

matches = times_pattern.findall(string)  # найти все совпадения

for time in matches:
  print(time)

09:00
13:00


#### 3. Вычленить операнды и оператор из строки: (`именованные группы`, `re-флаги` и `find-iter`)

In [85]:
regex = r"(-?\d+(?:\.\d+)?)\s*([-+*\/])\s*(-?\d+(?:\.\d+)?)"

Сложновато читать это выражение?

Добавим флаг `EXTENDED` и `MULTILINE` чтобы игнорировать проблеы и переводы строк!

In [86]:
# А для ясности добавим комменатрии
regex = r"""
(?P<first>-?\d+(?:\.\d+)?) # 1 операнд
\s*               # пробелы
(?P<operator>[-+*\/])         # оператор
\s*               # пробелы
(?P<second>-?\d+(?:\.\d+)?) # 2 операнд
"""

pattern = re.compile(regex, re.X | re.M)  # компилируем регулярное выражение

In [87]:
strings = [
  "1 + 2",
  "1.2 *3.4",
  "-3/ -6",
  "-2-2",
]

for string in strings:
  for m in pattern.finditer(string):
    print(m.groupdict())

{'first': '1', 'operator': '+', 'second': '2'}
{'first': '1.2', 'operator': '*', 'second': '3.4'}
{'first': '-3', 'operator': '/', 'second': '-6'}
{'first': '-2', 'operator': '-', 'second': '2'}


#### 4. Преобразовать дату (`.sub()`)

In [88]:
string = "April 15, 2003"
regex = r"(\w+) (\d+), (\d+)"

pattern = re.compile(regex, re.IGNORECASE)  # Игнорируем регистр
new_date = pattern.sub(r"\3, \1", string)

print(new_date)

2003, April


#### 5. Создать HTML (XML) дерево из переданной строки

[Stackoverflow](https://stackoverflow.com/a/1732454)

HTML невозможно распарсить с помощью регулярных выражений!

Дело в том что HTML - это **нерегулярный язык**!

Если на каком-то наборе данных регулярное выражение и будет работать, то на других наборах данных, в зависимости от реализации, может возникнуть бесконечный цикл или переполнение стека или всегда будет возвращаться некорректный результат и т.п.

Например, PHP и Python реагируют "ласково" на такие несоответсвия, а вот Java (OpenJDK 15.1) впадет в бесконечный цикл.